In [26]:
import scipy.io as sc
import numpy as np
from sklearn.svm import LinearSVC,SVC
import copy
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from HelperFunctions_ import classesInLabels

In [2]:
trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

In [3]:
#HYPSO test data 
sanfrancisco_data = plt.imread("../Training_data/sanfrancisco_2023-02-06_1809Z-bin3.png")
karachi_data =  plt.imread("../Training_data/karachi_2023-02-06_0531Z-bin3.png")
venice_data = plt.imread("../Training_data/venice_2023-02-07_0957Z-bin3.png")


In [4]:
print(len(sanfrancisco_data[0]))
print(len(sanfrancisco_data))

956
228


In [5]:
#Preprocess the labeling 

new_trondheim_gt = []
y = trondheim_gt.flatten()

for i in range(0,len(y),3): 
    new_trondheim_gt.append(y[i])

new_trondheim_gt = np.array(new_trondheim_gt)

In [6]:
def rotate_image(image): 
    rotated_image = []
    for i in range(len(image[0])): 
        x_array = []
        for j in range(len(image)-1,-1,-1):
            x_array.append(image[j][i])
        rotated_image.append(x_array)

    return np.array(rotated_image)

In [52]:
#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)
trondheim_new_labels[trondheim_new_labels == 10] = 9
trondheim_new_labels[trondheim_new_labels == 9] = 2

nl = [4,5,6,7,8]

for i in range(len(nl)):
        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3



In [53]:
new_trondheim = rotate_image(trondheim_data)

In [54]:
plt.imshow(trondheim_new_labels.reshape(956,228))
plt.axis('off')
plt.savefig("Tronhdeim_few_labels", dpi = 300, bbox_inches="tight")
plt.close()

In [55]:
X = np.array(new_trondheim).reshape((-1,3))
X = preprocessing.scale(X, axis=0)
y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [56]:
svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [57]:
predict = svm.predict(X) 

In [58]:
support = []
for i in range(3):
    support.append(np.mean(svm.support_vectors_[:,i]))

print(support)

[-0.07979147376751819, -0.08336550993168487, 0.024906638981792777]


In [62]:
support = (svm.support_vectors_)

In [82]:
prediction_array = []


for i in range(len(X)):
    prediction_pixel = [] 
    for j in range(len(X[0])): 
        pred = X[i][j]*support[1][j] - 0.12
        prediction_pixel.append(pred)
    if(np.mean(prediction_pixel) >= 0): 
        prediction_array.append(1)
    else: 
        prediction_array.append(0)

    #prediction_array.append(np.sign(np.mean(prediction_pixel)))
    
prediction_array = np.array(prediction_array)

In [83]:
plt.imshow(prediction_array.reshape(956,228))
plt.axis('off')
plt.savefig("Tronhdeim_predict", dpi = 300, bbox_inches="tight")
plt.close()